### load and preprocess

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from src.utils import inspect_nulls, pop_datetime
from src.preprocessing import (
    to_datetime,
    drop_full_nulls,
    try_drop_shared_nulls,
    drop_full_zero,
    normalize,
)

with open("data/energy_dataset.csv") as file:
    en = pd.read_csv(file, sep=",")
    en.name = "energy"

# create datetime column
en = to_datetime(en, "time")

# drop features with only zero values
en = drop_full_zero(en)

# nulls imputation
en = drop_full_nulls(en)
if en.datetime.diff().dt.total_seconds().value_counts().shape[0] == 1:
    en = en.interpolate(method="linear")
else:
    display("timestamps not evenly interpolate... chto delat?")
    #en = try_drop_shared_nulls(en, any_null=True)

# datetime as index
#en_norm.set_index(keys="datetime", inplace=True)

display(inspect_nulls(en))

### train/test splitting
at this stage:
* train/test only, w/o validation subset
* next contains previous training subset XOR non-overlapping subsets
* 80-20 ratio 

In [ ]:
train_test_split_ratio = 0.8
train_en = en[:round(en.shape[0]*train_test_split_ratio)]
test_en = en[round(en.shape[0]*train_test_split_ratio):]
display(train_en.shape, test_en.shape)

### batch and target

In [ ]:
features = ["total load actual"]
df = normalize(train_en[features])
df.describe()

In [ ]:
n_batches = 100
window = df.shape[0] // n_batches
batch_arrays = {key: None for key in df.columns}
batch_targets = {key: None for key in df.columns}
for feature in df.columns:
    arr = df[feature].to_numpy()
    batch_arrays[feature] = {
        batch_idx: arr[batch_idx * window : (batch_idx + 1) * window]
        for batch_idx in range(n_batches)
    }
    batch_targets[feature] = {batch_idx: arr[(batch_idx + 1) * window]
        for batch_idx in range(n_batches)
    }


In [ ]:
batch_targets